# 취향 분석

In [58]:
transaction = pd.read_json('./transaction_top.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]
transaction

,userId,newClothId,shopCnt
0,858063,1622018181681,1
1,858063,7054410169997,1
2,858064,1209634,1
3,858064,2066428259587,1
4,858064,2149254153363,2
...,...,...,...
1505320,1416946,1848166174682,1
1505321,1416946,6657950174519,2
1505322,1416947,2252678186665,1
1505323,1416948,1912952134031,2


In [59]:
cloth_meta = pd.read_json('./final_top.json')

In [60]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]
transaction

,userId,newClothId,shopCnt
0,858063,1622018181681,1
1,858063,7054410169997,1
3,858064,2066428259587,1
4,858064,2149254153363,2
6,858065,1841247138594,2
...,...,...,...
1505320,1416946,1848166174682,1
1505321,1416946,6657950174519,2
1505322,1416947,2252678186665,1
1505323,1416948,1912952134031,2


In [61]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))
a

,newClothId,idx
0,1622018181681,0
1,7054410169997,1
3,2066428259587,2
4,2149254153363,3
6,1841247138594,4
...,...,...
1505219,2126162235243,147721
1505267,2070504173827,147722
1505306,1894355139185,147723
1505314,1807072266056,147724


In [62]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")
transaction

,userId,newClothId,shopCnt,idx
0,858063,1622018181681,1,0
1,858063,7054410169997,1,1
2,858064,2066428259587,1,2
3,858064,2149254153363,2,3
4,858065,1841247138594,2,4
...,...,...,...,...
1478511,1416946,1848166174682,1,98
1478512,1416946,6657950174519,2,7605
1478513,1416947,2252678186665,1,147725
1478514,1416948,1912952134031,2,3963


In [116]:
user_meta = pd.read_json('./user_top.json')

In [117]:
user_meta = user_meta[favor_utop_column]
user_meta

,userId,size,bright,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858063,2.00,2.0,2.0,1.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,858064,1.25,1.0,1.0,2.750000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0,0.666667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333
2,858065,1.60,1.8,2.0,2.200000,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.750000,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,858066,2.00,2.0,2.0,2.000000,0.666667,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333
4,858067,1.75,2.0,2.0,1.500000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.500000,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558881,1416944,2.00,2.0,2.0,2.000000,0.333333,0.166667,0.0,0.333333,0.166667,0.000000,0.000000,0.0,0.0,0.166667,0.5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000
558882,1416945,1.50,1.5,1.5,2.000000,0.500000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000
558883,1416946,2.00,2.0,2.0,2.166667,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0,0.166667,0.0,0.416667,0.0,0.0,0.0,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.083333
558884,1416947,1.00,2.0,2.0,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [118]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,size,bright,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858063,2.00,2.0,2.0,1.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,858064,1.25,1.0,1.0,2.750000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0,0.666667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333
2,858065,1.60,1.8,2.0,2.200000,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.750000,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,858066,2.00,2.0,2.0,2.000000,0.666667,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333
4,858067,1.75,2.0,2.0,1.500000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.500000,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553548,1416944,2.00,2.0,2.0,2.000000,0.333333,0.166667,0.0,0.333333,0.166667,0.000000,0.000000,0.0,0.0,0.166667,0.5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000
553549,1416945,1.50,1.5,1.5,2.000000,0.500000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000
553550,1416946,2.00,2.0,2.0,2.166667,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0,0.166667,0.0,0.416667,0.0,0.0,0.0,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.083333
553551,1416947,1.00,2.0,2.0,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [119]:
user_meta

,userId,size,bright,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858063,2.00,2.0,2.0,1.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,858064,1.25,1.0,1.0,2.750000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0,0.666667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333
2,858065,1.60,1.8,2.0,2.200000,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.750000,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,858066,2.00,2.0,2.0,2.000000,0.666667,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333
4,858067,1.75,2.0,2.0,1.500000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.500000,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553548,1416944,2.00,2.0,2.0,2.000000,0.333333,0.166667,0.0,0.333333,0.166667,0.000000,0.000000,0.0,0.0,0.166667,0.5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000
553549,1416945,1.50,1.5,1.5,2.000000,0.500000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000
553550,1416946,2.00,2.0,2.0,2.166667,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0,0.166667,0.0,0.416667,0.0,0.0,0.0,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.083333
553551,1416947,1.00,2.0,2.0,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [120]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 160001
new_user.loc[new_user_idx, ('size')] = 0
new_user.loc[new_user_idx, ('bright')] = 1
new_user.loc[new_user_idx, ('color')] = 2
new_user.loc[new_user_idx, ('thickness')] = 2
new_user.loc[new_user_idx, ('smallCategoryHalfshort')] = 0
new_user.loc[new_user_idx, ('smallCategoryShirt')] = 0.5
new_user.loc[new_user_idx, ('smallCategoryCollar')] = 0
new_user.loc[new_user_idx, ('smallCategoryHoody')] = 0.3
new_user.loc[new_user_idx, ('smallCategorySweatshirt')] = 0
new_user.loc[new_user_idx, ('smallCategoryKnit')] = 0
new_user.loc[new_user_idx, ('smallCategoryLong')] = 0.2
new_user.loc[new_user_idx, ('smallCategoryShort')] = 0
new_user.loc[new_user_idx, ('smallCategoryOthers')] = 0
new_user.loc[new_user_idx, ('colorWhite')] = 0.4
new_user.loc[new_user_idx, ('colorGrey')] = 0
new_user.loc[new_user_idx, ('colorBlack')] = 0.4
new_user.loc[new_user_idx, ('colorRed')] = 0
new_user.loc[new_user_idx, ('colorPink')] = 0.2
new_user.loc[new_user_idx, ('colorOrange')] = 0
new_user.loc[new_user_idx, ('colorIvory')] = 0
new_user.loc[new_user_idx, ('colorYellow')] = 0
new_user.loc[new_user_idx, ('colorGreen')] = 0
new_user.loc[new_user_idx, ('colorBlue')] = 0
new_user.loc[new_user_idx, ('colorPurple')] = 0
new_user.loc[new_user_idx, ('colorBrown')] = 0
new_user.loc[new_user_idx, ('colorBeige')] = 0
new_user.loc[new_user_idx, ('colorJean')] = 0
new_user.loc[new_user_idx, ('colorPattern')] = 0
new_user.loc[new_user_idx, ('colorOthers')] = 0
new_user = new_user[['userId', 'size', 'bright', 'color', 'thickness', 'smallCategoryHalfshort', 'smallCategoryShirt',
       'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit', 'smallCategoryLong', 'smallCategoryShort',
       'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']]
new_user

,userId,size,bright,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,160001.0,0.0,1.0,2.0,2.0,0.0,0.5,0.0,0.3,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [150]:
import random
unique_user_id = sorted(list(sub_user.userId.unique()))
user_list = random.sample(unique_user_id, 200000)
len(user_list)

200000

In [151]:
sub_user=sub_user.loc[user_meta.userId.isin(user_list),:]
sub_user = sub_user.reset_index(drop=True)
sub_user

,userId,size,bright,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858063,2.000000,2.000000,2.0,1.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
1,858067,1.750000,2.000000,2.0,1.500000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.500000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.250000
2,858068,2.000000,2.000000,2.0,2.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
3,858069,2.000000,2.000000,1.0,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
4,858073,1.000000,2.000000,2.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1416925,1.882353,2.000000,2.0,2.117647,0.235294,0.352941,0.0,0.058824,0.000000,0.000000,0.352941,0.000000,0.0,0.117647,0.000000,0.176471,0.0,0.0,0.0,0.117647,0.117647,0.235294,0.117647,0.0,0.000000,0.117647,0.0,0.0,0.000000
199996,1416929,1.666667,1.666667,1.0,2.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.333333,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.333333
199997,1416933,2.000000,2.142857,2.0,1.857143,0.285714,0.000000,0.0,0.285714,0.142857,0.000000,0.000000,0.285714,0.0,0.142857,0.285714,0.285714,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.142857
199998,1416936,1.333333,1.666667,2.0,1.333333,0.000000,0.333333,0.0,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.666667,0.000000,0.0,0.0,0.000000


In [152]:
new_user['colorWhite'][0]

0.4

In [153]:
column_list = ['colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']
col_list = []
for i in column_list:
    if new_user[i][0] > 0.2:
        col_list.append(i)

In [154]:
import time
start = time.time()

for i in range(len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)
finish = time.time()
print(finish - start)

5.430176496505737


In [155]:
col_list

['colorWhite', 'colorBlack']

In [156]:
sub_user.insert(3, 'favorSum', 0, True)

sub_user.favorSum = sum(sub_user[col_i] for col_i in col_list) 

In [157]:
sub_user

,userId,size,bright,favorSum,color,thickness,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers
0,858063,2.000000,2.000000,0.500000,2.0,1.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.400000,0.500000,0.100000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
1,858067,1.750000,2.000000,0.800000,2.0,1.500000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.000000,0.0,0.400000,0.000000,0.400000,0.0,0.0,0.0,0.500000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.250000
2,858068,2.000000,2.000000,0.800000,2.0,2.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.400000,0.000000,0.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
3,858069,2.000000,2.000000,0.800000,1.0,2.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.400000,0.000000,0.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
4,858073,1.000000,2.000000,0.800000,2.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.400000,0.000000,0.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1416925,1.882353,2.000000,0.505882,2.0,2.117647,0.235294,0.352941,0.0,0.058824,0.000000,0.000000,0.352941,0.000000,0.0,0.282353,0.000000,0.223529,0.0,0.0,0.0,0.117647,0.117647,0.235294,0.117647,0.0,0.000000,0.117647,0.0,0.0,0.000000
199996,1416929,1.666667,1.666667,0.133333,1.0,2.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.066667,0.000000,0.066667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.333333
199997,1416933,2.000000,2.142857,0.371429,2.0,1.857143,0.285714,0.000000,0.0,0.285714,0.142857,0.000000,0.000000,0.285714,0.0,0.257143,0.285714,0.114286,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.142857
199998,1416936,1.333333,1.666667,0.800000,2.0,1.333333,0.000000,0.333333,0.0,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,0.400000,0.000000,0.400000,0.0,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.666667,0.000000,0.0,0.0,0.000000


In [158]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('favorSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
real_user

[885503, 911393, 1221136]

In [207]:
# 사이즈
# 여성 남성은 확실하게 나눠서 비교
# 각 파트에서 userHeight, userWeight가 같은 사람이 있을 것!
# 그걸 조회하는 별로 안걸림
# random으로 1개를 선택

# 취향
# 이게 문제

In [16]:
cloth_meta = cloth_meta[favor_ctop_column]
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale
0,1929966,1,0,0.0,0,0.0,0,1.0,0.0
1,1927658,1,0,0.0,0,0.0,0,1.0,0.0
2,1927660131403,1,0,4.5,2,172.0,70,1.0,0.0
3,1928209,0,1,5.0,1,0.0,0,0.0,1.0
4,1927270,1,0,0.0,0,0.0,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
249601,1413008,1,0,0.0,0,0.0,0,1.0,0.0
249602,779965,0,1,0.0,0,0.0,0,0.0,1.0
249603,1353489,1,0,0.0,0,0.0,0,1.0,0.0
249604,1607233,0,1,0.0,0,0.0,0,0.0,1.0


In [17]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,1929966,1,0,0.0,0,0.0,0,1.0,0.0,NaN
1,1927658,1,0,0.0,0,0.0,0,1.0,0.0,NaN
2,1927660131403,1,0,4.5,2,172.0,70,1.0,0.0,112928.0
3,1928209,0,1,5.0,1,0.0,0,0.0,1.0,NaN
4,1927270,1,0,0.0,0,0.0,0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
249601,1413008,1,0,0.0,0,0.0,0,1.0,0.0,NaN
249602,779965,0,1,0.0,0,0.0,0,0.0,1.0,NaN
249603,1353489,1,0,0.0,0,0.0,0,1.0,0.0,NaN
249604,1607233,0,1,0.0,0,0.0,0,0.0,1.0,NaN


In [18]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

147726

In [19]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,1927660131403,1,0,4.5,2,172.000000,70,1.0,0.0,112928.0
1,1927703131473,1,0,4.5,2,165.000000,51,1.0,1.0,118965.0
2,1928015131481,1,0,5.0,1,165.000000,55,1.0,1.0,80551.0
3,1929956131505,1,0,5.0,1,172.000000,70,1.0,0.0,72010.0
4,1929608131565,1,0,5.0,2,175.000000,64,1.0,0.0,125441.0
...,...,...,...,...,...,...,...,...,...,...
147721,9519160268389,1,0,5.0,1,160.500000,49,1.0,1.0,104680.0
147722,1275799268869,1,0,5.0,1,165.600000,58,0.4,0.6,94760.0
147723,1241388268878,1,0,4.6,5,176.333333,65,1.0,0.0,129703.0
147724,1241388268879,1,0,4.6,5,164.000000,54,0.5,0.5,121484.0


In [20]:
cloth_meta.idx = cloth_meta.idx.astype('int')
cloth_meta

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,1927660131403,1,0,4.5,2,172.000000,70,1.0,0.0,112928
1,1927703131473,1,0,4.5,2,165.000000,51,1.0,1.0,118965
2,1928015131481,1,0,5.0,1,165.000000,55,1.0,1.0,80551
3,1929956131505,1,0,5.0,1,172.000000,70,1.0,0.0,72010
4,1929608131565,1,0,5.0,2,175.000000,64,1.0,0.0,125441
...,...,...,...,...,...,...,...,...,...,...
147721,9519160268389,1,0,5.0,1,160.500000,49,1.0,1.0,104680
147722,1275799268869,1,0,5.0,1,165.600000,58,0.4,0.6,94760
147723,1241388268878,1,0,4.6,5,176.333333,65,1.0,0.0,129703
147724,1241388268879,1,0,4.6,5,164.000000,54,0.5,0.5,121484


In [21]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]
transaction_source

[(858063, 0),
 (858063, 1),
 (858064, 2),
 (858064, 3),
 (858065, 4),
 (858065, 5),
 (858065, 6),
 (858066, 7),
 (858066, 8),
 (858066, 9),
 (858067, 10),
 (858067, 11),
 (858067, 12),
 (858068, 13),
 (858069, 14),
 (858070, 15),
 (858071, 16),
 (858071, 17),
 (858071, 18),
 (858072, 19),
 (858073, 20),
 (858074, 21),
 (858075, 22),
 (858075, 23),
 (858075, 24),
 (858076, 25),
 (858077, 26),
 (858077, 27),
 (858077, 28),
 (858078, 29),
 (858079, 30),
 (858080, 31),
 (858080, 32),
 (858080, 33),
 (858080, 34),
 (858081, 35),
 (858081, 36),
 (858081, 37),
 (858082, 38),
 (858082, 39),
 (858082, 40),
 (858083, 41),
 (858083, 42),
 (858084, 43),
 (858085, 44),
 (858086, 45),
 (858086, 46),
 (858087, 47),
 (858087, 48),
 (858088, 49),
 (858088, 50),
 (858089, 51),
 (858090, 52),
 (858090, 53),
 (858090, 54),
 (858090, 55),
 (858090, 56),
 (858091, 57),
 (858091, 58),
 (858091, 59),
 (858092, 60),
 (858092, 61),
 (858092, 62),
 (858092, 63),
 (858093, 64),
 (858093, 65),
 (858094, 66),
 (858

In [22]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 553553
clothId의 유니크 개수 : 147726
user테이블의 userId의 유니크 개수 : 553553
cloth테이블의 clothId의 유니크 개수 : 147726


In [23]:
sum(transaction.idx.isna())

0

In [43]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]
user_features_source

[(858063, [1, 0, 163, 51]),
 (858064, [1, 0, 183, 90]),
 (858065, [0, 1, 164, 57]),
 (858066, [1, 0, 175, 55]),
 (858067, [0, 1, 172, 59]),
 (858068, [1, 0, 178, 74]),
 (858069, [1, 0, 178, 82]),
 (858070, [1, 0, 167, 70]),
 (858071, [1, 0, 177, 68]),
 (858072, [0, 1, 160, 50]),
 (858073, [1, 0, 160, 42]),
 (858074, [0, 1, 156, 47]),
 (858075, [1, 0, 167, 65]),
 (858076, [1, 0, 177, 80]),
 (858077, [1, 0, 178, 77]),
 (858078, [1, 0, 186, 82]),
 (858079, [1, 0, 173, 66]),
 (858080, [0, 1, 170, 59]),
 (858081, [0, 1, 162, 51]),
 (858082, [1, 0, 190, 78]),
 (858083, [0, 1, 158, 55]),
 (858084, [1, 0, 180, 90]),
 (858085, [1, 0, 175, 70]),
 (858086, [0, 1, 171, 60]),
 (858087, [0, 1, 163, 56]),
 (858088, [1, 0, 178, 56]),
 (858089, [0, 1, 156, 50]),
 (858090, [1, 0, 170, 68]),
 (858091, [0, 1, 163, 50]),
 (858092, [1, 0, 172, 67]),
 (858093, [1, 0, 186, 68]),
 (858094, [0, 1, 160, 55]),
 (858095, [1, 0, 158, 57]),
 (858096, [0, 1, 167, 45]),
 (858097, [1, 0, 173, 70]),
 (858098, [0, 1, 179

In [25]:
# cloth_features_source = [(cloth_meta['idx'][i],
#                         [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i], cloth_meta['clothRate'][i], cloth_meta['clothReviewCnt'][i],
#                          cloth_meta['userHeight'][i], cloth_meta['userWeight'][i], cloth_meta['userMale'][i], cloth_meta['userFemale'][i], ]) for i in range(cloth_meta.shape[0])]
# cloth_features_source

In [26]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]
cloth_features_source

[(112928, [1, 0, 172.0, 70]),
 (118965, [1, 0, 165.0, 51]),
 (80551, [1, 0, 165.0, 55]),
 (72010, [1, 0, 172.0, 70]),
 (125441, [1, 0, 175.0, 64]),
 (140816, [1, 0, 175.5714285714, 78]),
 (143508, [1, 0, 178.0, 71]),
 (62704, [1, 0, 181.0, 72]),
 (114189, [1, 1, 172.14, 66]),
 (112127, [1, 1, 169.0, 60]),
 (128647, [1, 1, 178.8666666667, 79]),
 (128649, [1, 1, 169.75, 59]),
 (19089, [1, 1, 167.0, 63]),
 (72119, [0, 1, 163.0, 65]),
 (48682, [0, 1, 171.0, 76]),
 (127193, [1, 0, 175.0, 80]),
 (139966, [1, 0, 179.0909090909, 72]),
 (25192, [1, 0, 163.6, 56]),
 (22943, [1, 0, 169.6666666667, 64]),
 (26237, [1, 0, 166.5, 59]),
 (91076, [1, 0, 160.0, 57]),
 (128063, [1, 0, 165.0, 80]),
 (32604, [1, 0, 175.0, 57]),
 (59834, [0, 1, 173.0, 73]),
 (20152, [0, 1, 161.0, 50]),
 (47259, [1, 0, 184.0, 84]),
 (74412, [1, 0, 171.0, 65]),
 (95118, [1, 0, 177.0, 65]),
 (13699, [1, 0, 162.6363636364, 51]),
 (99673, [0, 1, 164.0, 47]),
 (128767, [0, 1, 176.6666666667, 68]),
 (96280, [0, 1, 173.4166666667, 

In [44]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [45]:
interactions, weights = dataset.build_interactions(transaction_source)

In [46]:
user_features = dataset.build_user_features(user_features_source)

In [47]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [48]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [49]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [287]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

In [30]:
best_params

{'learning_rate': 0.04958125359730216, 'no_components': 2}

In [31]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.05 2


In [50]:
learning_rate = 0.05
no_components = 10

In [51]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [01:31<00:00,  9.18s/it]


In [52]:
# 모델 저장하기
import pickle
with open('model_size_top.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [54]:
# 모델 불러오기
import pickle
with open('model_size_top.pickle', 'rb') as f:
    model_top = pickle.load(f)

In [55]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [57]:
rec_size_top = sample_recommendation(model_top, cloth_meta.idx, real_user)
rec_size_top

[1903814132550,
 1983415132560,
 1938490132348,
 1749614132435,
 1872454131910,
 2139144142785,
 2174419133241,
 1793413134207,
 1907926131751,
 1496186131886]